In [1]:
# @title Default title text

years = list(range(1991,2022))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

import requests
import time

from google.colab import drive
drive.mount('/content/gdrive')
# format the url and use requests to retrieve and store html
# data from each url in loop
for year in years:
  url = url_start.format(year)
  time.sleep(15)
  data = requests.get(url)

  with open("mvp/{}.html".format(year), "w+") as f:
    f.write(data.text)


from bs4 import BeautifulSoup

# Single year test
with open("mvp/1991.html") as f:
  page = f.read()

soup = BeautifulSoup(page, "html.parser")
soup.find('tr', class_="over_header").decompose()
mvp_table = soup.find(id="mvp")

import pandas as pd

mvp_1991 = pd.read_html(str(mvp_table))[0]



# for each html file parse the file to find the tables with the id for mvp then convert it into a
# dataframe with pandas and append that to a list of dataframes then convert to csv
dfs = []
for year in years:

  with open("mvp/{}.html".format(year)) as f:
   page = f.read()
  soup = BeautifulSoup(page, "html.parser")
  soup.find('tr', class_="over_header").decompose()
  mvp_table = soup.find(id="mvp")
  mvp = pd.read_html(str(mvp_table))[0]
  mvp["Year"] = year
  dfs.append(mvp)

mvps = pd.concat(dfs)

mvps.head()

mvps.to_csv("mvps.csv")


## Repeat above for player stats in each year
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

url = player_stats_url.format(1991)
data = requests.get(url)
with open("player/1991.html".format(year), "w+") as f:
   f.write(data.text)

# Connect driver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()

options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
options.add_argument('--remote-debugging-port=9222')  # Use a specific port for debugging
options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration

service = Service()
driver = webdriver.Chrome(service=service, options=options)

# single year test
year = 1991

url = player_stats_url.format(year)
driver.get(url)
driver.execute_script("window.scrollTo(1,10000)")
time.sleep(2)

html = driver.page_source

with open("player/{}.html".format(year), "w+") as f:
  f.write(html)

# Player stats table can fully load on the driver, so the player page's html
# can be stored in a loop
for year in years:
  url = player_stats_url.format(year)
  driver.get(url)
  driver.execute_script("window.scrollTo(1,10000)")
  time.sleep(10)

  html = driver.page_source
  with open("player/{}.html".format(year), "w+") as f:
    f.write(html)

# for each html parse the file to find the table with the id for per game stats then
# then convert it into a dataframe with pandas and append that to a list of dataframes then convert to csv
dfs = []
for year in years:
 with open("player/{}.html".format(year)) as f:
  page = f.read()

 soup = BeautifulSoup(page, "html.parser")
 soup.find('tr', class_="thead").decompose()
 player_table = soup.find(id="per_game_stats")
 player = pd.read_html(str(player_table))[0]
 player["Year"] = year
 dfs.append(player)



players = pd.concat(dfs)
players.to_csv("Player.csv")

# format the url and use requests to retrieve and store html
# data from each url in loop
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for year in years:
 url = team_stats_url.format(year)
 data = requests.get(url)
 time.sleep(10)
 with open("team/{}.html".format(year), "w+") as f:
  f.write(data.text)

# for each html parse the file to find the table with the id for divs_standings then
# then convert it into a dataframe with pandas and append that to a list of dataframes then convert to csv
dfs = []
for year in years:
  with open("team/{}.html".format(year)) as f:
    page = f.read()
  soup = BeautifulSoup(page, "html.parser")
  soup.find('tr', class_="thead").decompose()
  team_table = soup.find(id="divs_standings_E")
  team = pd.read_html(str(team_table))[0]
  team["Year"] = year
  team["Team"] = team["Eastern Conference"]
  del team["Eastern Conference"]
  dfs.append(team)

  soup = BeautifulSoup(page, "html.parser")
  soup.find('tr', class_="thead").decompose()
  team_table = soup.find(id="divs_standings_W")
  team = pd.read_html(str(team_table))[0]
  team["Year"] = year
  team["Team"] = team["Western Conference"]
  del team["Western Conference"]
  dfs.append(team)
teams = pd.concat(dfs)

teams.to_csv("teams.csv")


NameError: name 'players' is not defined